In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from bs4 import BeautifulSoup

from_date = "01/01/2021"
to_date = "30/01/2021"

driver = webdriver.Chrome()
url = "https://online.whittlesea.vic.gov.au/s/publicregister"
driver.get(url)

wait = WebDriverWait(driver, 10)
from_date_input = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div/div/div/div/div[3]/div/div/div/c-search-public-register/lightning-layout/slot/lightning-layout-item[4]/slot/lightning-layout/slot/lightning-layout-item[1]/slot/lightning-input/lightning-datepicker/div[1]/div/input")))
to_date_input = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div/div/div/div/div[3]/div/div/div/c-search-public-register/lightning-layout/slot/lightning-layout-item[4]/slot/lightning-layout/slot/lightning-layout-item[2]/slot/lightning-input/lightning-datepicker/div[1]/div/input")))

from_date_input.clear()
from_date_input.send_keys(from_date)

to_date_input.clear()
to_date_input.send_keys(to_date)

search_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div/div/div/div/div[3]/div/div/div/c-search-public-register/lightning-layout/slot/lightning-layout-item[5]/slot/lightning-layout/slot/lightning-layout-item[1]")))
search_button.click()

extracted_data = []

while True:
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//table[@role='grid']")))
        
        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')

        rows = soup.select("table[role='grid'] tbody tr")

        for row in rows:
            application_number = row.select_one("th div lightning-base-formatted-text").text.strip()
            application_type = row.select_one("td[data-label='Application Type'] div lightning-base-formatted-text").text.strip()
            parent_application_number = row.select_one("td[data-label='Parent Application Number'] div lightning-base-formatted-text").text.strip()
            submitted_date = row.select_one("td[data-label='Submitted Date'] div lightning-formatted-date-time").text.strip()
            property_address = row.select_one("td[data-label='Property Address'] div lightning-base-formatted-text").text.strip()

            extracted_data.append({
                "Application Number": application_number,
                "Application Type": application_type,
                "Parent Application Number": parent_application_number,
                "Submitted Date": submitted_date,
                "Property Address": property_address
            })

        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@title='Next Page']")))
        
        if "slds-button_disabled" in next_button.get_attribute("class"):
            break
        
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        
        next_button.click()
        
        time.sleep(3)
    
    except ElementClickInterceptedException:
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CLASS_NAME, "slds-spinner_container")))
    except TimeoutException:
        print("Timeout: Next button not found within expected time.")
        break

df = pd.DataFrame(extracted_data)

print(df)

df.to_excel("output_data_02.xlsx", index=False)

driver.quit()


Timeout: Next button not found within expected time.
    Application Number              Application Type  \
0            PLN-36746           Subdivision of Land   
1            PLN-36744            SPEARCertification   
2            PLN-36743           Subdivision of Land   
3            PLN-36742            SPEARCertification   
4            PLN-36741           Subdivision of Land   
..                 ...                           ...   
105           714827/4  Multi Unit (2 – 4 dwellings)   
106           719844/1                      Building   
107           719843/1                 Change of Use   
108           719841/1                      Building   
109           719840/1                      Building   

    Parent Application Number Submitted Date  \
0                                 29/01/2021   
1                                 22/01/2021   
2                                 22/01/2021   
3                                 22/01/2021   
4                                 